## RFM Analizi ile Müşteri Segmentasyonu - Online Retail II (2010-2011)

[Veri Bilimci Yetiştirme Programı](bootcamp.veribilimiokulu.com/egitim/veri-bilimci-yetistirme-programi/)'nda ele alınan RFM Analizi konusuna ilişkin müşteri segmentasyon örneği. İş problemine, verinin hikayesine ve değişken bilgilerine aşağıdaki sunumdan ulaşabilirsiniz.

<iframe src="//www.slideshare.net/slideshow/embed_code/key/tqRmlGIqyAkL6u" width="595" height="485" frameborder="0" marginwidth="0" marginheight="0" scrolling="no" style="border:1px solid #CCC; border-width:1px; margin-bottom:5px; max-width: 100%;" allowfullscreen> </iframe> <div style="margin-bottom:5px"> <strong> <a href="//www.slideshare.net/CemalCici/dsmlbc-rfm-analizi-ile-mteri-segmentasyonu" title="DSMLBC RFM ANALİZİ İLE MÜŞTERİ SEGMENTASYONU" target="_blank">DSMLBC RFM ANALİZİ İLE MÜŞTERİ SEGMENTASYONU</a> </strong> from <strong><a href="//www.slideshare.net/CemalCici" target="_blank">Cemal Cici</a></strong> </div>

## Gerekli Kütüphanelerin Yüklenmesi

In [ ]:
import pandas as pd # Veri Manipülasyonu
import numpy as np # Matris İşlemleri
import datetime as dt # Tarih Saat İşlemleri
import plotly.express as px
import warnings # Uyarılar


warnings.filterwarnings("ignore") # Uyarıları görmezden geldik.
pd.set_option('display.expand_frame_repr', False)

## Veri Setinin Yüklenmesi

In [ ]:
online_retail_2010_2011_df = pd.read_csv("../input/online-retail-ii-uci-two-peroid/online_retail_II_2010_2011.csv", sep=";")

In [ ]:
df = online_retail_2010_2011_df.copy()

In [ ]:
df.head()

## Verinin Yapısal Olarak Hazırlanması

In [ ]:
df.shape

Çıktı incelendiğinde 2010-2011 yılına ait **541.910** işlem kaydı görülmektedir.

In [ ]:
df.isnull().sum()

Tablo incelendiğinde `Description` ve `Customer ID` değişkenlerinde eksik değer görülmektedir. Analiz müşteri hareketlerini kapsadığından dolayı eksik değer sahip gözlemleri silindi.

In [ ]:
df.dropna(inplace=True)
df.isnull().sum()

Tablo incelendiğinde eksik gözlemlerin silindiği görülmektedir.

In [ ]:
df = df[~(df["Invoice"].str.contains("C"))]
df.shape

Analiz sırasında satın almalarla ilgilendiğimizden içerisinde iade işlemi barındran gözlemleri çıkardık. Eksik değere bağlı gözlemleri ve iade işlemleri barındıran gözlemleri çıkarttığımızda analiz edilecek **397.925** gözlemimiz bulunmaktadır.

In [ ]:
df["Customer ID"] = df["Customer ID"].astype(int).astype(str)
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df.dtypes

Analiz esnasında çeşitli yapısal problemler yaşamamak adına müşteri numaraları kategorik değişkene, fatura tarihleri ise tarih-saat değişkenine dönüştürüldü. 

## Keşifçi Veri Analizi

In [ ]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.80, 0.90, 0.95, 0.99]).T

Tablo incelendiğinde aykırı değerler görülmektedir. Analiz müşteri hareketlerini kapsadığından dolayı aykırı değer analizi yapmayı tercih edlimedi.

In [ ]:
for stockcode in list(df["StockCode"].value_counts().index[:10]):
    print(stockcode, "\n", df.loc[df["StockCode"] == stockcode, "Description"].value_counts(), end="\n\n")

Çıktı incelendiğinde ürün kodlarına karşılık gelen açıklamalar değişmektedir. Bazı ürün kodları birden fazla açıklamaya denk gelirken bazı ürün kodları birden fazla ürüne denk gelmektedir. Uzun vadede bu kısmın incelenebilir.

In [ ]:
df.groupby("Country").agg({"Invoice": "nunique",
                           "Quantity": "sum",
                           "Price": "sum"}).sort_values("Invoice", ascending=False).round(2).head()

Tablo incelendiğinde en çok alışveriş `United Kingdom` ülkesinde yapılmış olup **16.649** alışveriş yapılmıştır. Söz konusu ülkeye toplamda **4.269.472** ürün satılmış ve toplamda **1.050.205,16** sterlin değerinde ürünler satılmıştır.

In [ ]:
df["TotalPrice"] = df["Quantity"] * df["Price"]
df.groupby("Invoice").agg({"TotalPrice": "sum"}).sort_values("TotalPrice", ascending=False).round(2).head()

Tablo incelendiğinde bize en çok para kazandıran işlem **168.469,60** sterlin değerinde `581483` numaralı fatura olduğu görülmektedir.

## RFM Analizi

### RFM Tablosunun Oluşturulması

In [ ]:
def create_rfm_table(dataframe:pd.DataFrame, dataframe_id:str, rfm_grid:dict, segment_list=False):
    """
    RFM Analizi için tablonun oluşturulmasını sağlayan fonksiyon.
    Bu fonksiyon işlemleri 4 adımda tamamlamaktadır.
        1. Adım: RFM metriklerinin oluşturulması
            RFM metriklerini oluştururken gruplanacak id değerini ve R-F-M değerlerini kullanıcı kendi belirler.
        2. Adım: RFM skorlarının oluşturulması.
            RFM skorları oluşturulurken metrik tablosundan yararlanılır. Değerler 0-20, 21-40, 41-60, 61-80 ve 81-100 yüzdeliklerine göre 5 parçaya bölünmüştür. 
            Frequency'de iki yüzdelik aralığı arasında kalan değerlerin ilkine etiket atalamak adına rank() metodu kullanılmıştır.
        3. Adım: Segmentlerin oluşturulması.
            R ve F skorlarına göre RegEx yapısı kullanılarak segmentler oluşturulmuştur.
            Referans alınan kaynak: https://guillaume-martin.github.io/rfm-segmentation-with-python.html
    
    Parameters
    -----------
    dataframe pd.DataFrame
        RFM tablosunun oluşması için gereken veri yapısı
    dataframe_id str
        Probleme bağlı olarak ele alınacak id sütunu
    rfm_grid dict
        Probleme bağlı olara kullanılacak toplulaştırma sözlüğü
    
    Returns
    -----------
    rfm_table pd.DataFrame
        Oluşturulan RFM tablosu.
    seg_map.values() list
        Segment listesi. 
    
    Examples
    -----------
    >>> ...
    >>> analyse_date = dt.datetime(2011, 12, 11)
    >>> agg_dict = {"InvoiceDate": lambda date: (today_date - date.max()).days,
                    "Invoice": "nunique",
                    "TotalPrice": "sum"}
    >>> rfm = create_rfm_table(df, "Customer ID", agg_dict)
    >>> rfm.head()
         Customer ID  Recency  Frequency  Monetary recency_score frequency_score monetary_score RFM_SCORE       segment
    0       12346      326          1  77183.60             1               1              5        11      hibernating
    1       12347       40          7   4310.00             4               5              5        45  loyal_customers
    2       12348       76          4   1797.24             3               4              4        34  loyal_customers
    3       12349       19          1   1757.55             4               1              4        41        promising
    4       12350      311          1    334.40             1               1              2        11      hibernating
    """
    # Adım 1: RFM Metriklerinin oluşturulması
    rfm_table = dataframe.groupby(dataframe_id).agg(rfm_grid)
    rfm_table.columns = ["Recency", "Frequency", "Monetary"]
    rfm_table = rfm_table[rfm_table["Monetary"] > 0]
    rfm_table.reset_index(inplace=True)
    
    # Adım 2: RFM Skorlarının oluşturulması
    rfm_table["recency_score"] = pd.qcut(rfm_table['Recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm_table["frequency_score"] = pd.qcut(rfm_table['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm_table["monetary_score"] = pd.qcut(rfm_table['Monetary'], 5, labels=[1, 2, 3, 4, 5])
    
    # Adım 3: Segmentlerin oluşturulması
    rfm_table["RFM_SCORE"] = (rfm_table['recency_score'].astype(str) + rfm_table['frequency_score'].astype(str))
    seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
    }
    rfm_table['segment'] = rfm_table['RFM_SCORE'].replace(seg_map, regex=True)
    
    
    # Adım 4: RFM tablosunun döndürülmesi
    if segment_list:        
        return rfm_table, seg_map.values() 
    return rfm_table

In [ ]:
analyse_date = dt.datetime(2011, 12, 11)
agg_dict = {"InvoiceDate": lambda date: (analyse_date - date.max()).days,
            "Invoice": "nunique",
            "TotalPrice": "sum"}
rfm, list_segment = create_rfm_table(df, "Customer ID", agg_dict, segment_list=True)
rfm.head()

Söz konusu problem için RFM tablosunu oluşturulurken aşağıdaki hususlar dikkate alınmıştır:

* Analiz günü 11/12-2011 olarak belirlenmiştir.
* Recency değeri için müşterinin son alışveriş günününü hesapladık.
* Frequency değeri için müşterinin eşsiz fatura sayısını hesapladık.
* Monetary değeri için müşterinin toplam bıraktığı parayı hesapladık.

`12346` müşterisi incelendiğinde, kişi en son **326** gün alışveriş yapmış, analiz gününe kadar **1** kez alışveriş yapmış ve analiz gününe kadar toplamda bize **77.183,60** sterlin para kazandırarak **hibernating** segmentinde sınıflandırılmıştır. 

### Segment İstatistikleri

In [ ]:
def create_segment_statistic(rfm_table):
    """
    RFM tablosu sonucunda elde ettiğimiz segmentlerin R-F-M metriklerine göre ortalama değerlerinin alındığı ve segment sayılarının tablolaştırıldığı fonksiyondur.
    
    Parameters
    -----------
    rfm_table pd.DataFrame
        create_rfm_table() fonksiyonu ile oluşturulan RFM tablosu
    
    Returns
    -----------
    segment_statistics pd.DataFrame
        RFM metriklerinin ortalamasının, segment sayıların ve kümülatif oranların oluşturulduğu tablo.
    
    Examples
    -----------
    >>> ...
    >>> analyse_date = dt.datetime(2011, 12, 11)
    >>> agg_dict = {"InvoiceDate": lambda date: (today_date - date.max()).days,
                    "Invoice": "nunique",
                    "TotalPrice": "sum"}
    >>> rfm = create_rfm_table(df, "Customer ID", agg_dict)
    >>> seg_stat = create_segment_statistic(rfm)
    >>> seg_stat
                   segment  Recency  Frequency  Monetary  seg_count  ratio  cum_ratio
    0          hibernating   244.01       1.10    468.18       1069  24.64      24.64
    1      loyal_customers    42.17       6.82   3104.73        787  18.14      42.78
    2            champions     9.30      11.72   6288.26        665  15.33      58.11
    3              at_risk   171.67       2.83   1201.91        588  13.55      71.67
    4  potential_loyalists    20.34       2.03    846.57        478  11.02      82.69
    5       about_to_sleep    64.07       1.15    503.07        339   7.81      90.50
    6       need_attention    63.88       2.39   1087.42        196   4.52      95.02
    7            promising    28.71       1.00    327.78         96   2.21      97.23
    8           cant_loose   134.21       8.24   3551.16         68   1.57      98.80
    9        new_customers    11.15       1.00    342.55         52   1.20     100.00
    """
    # Adım 1: Segmentlere göre RFM metriklerinin ortalama değerlerinin alınması.
    segment_table = rfm_table[["Recency", "Frequency", "Monetary", "segment"]].groupby("segment").agg(["mean"]).reset_index()
    segment_table.columns = segment_table.columns.droplevel(1)
    
    # Adım 2: Segmentlerin sayılarının oluşturulması.
    count_table = rfm_table["segment"].value_counts()
    count_table = pd.DataFrame({"segment": count_table.index, "seg_count": count_table.values})
    
    # Adım 3: Adım 1'de ve Adım 2'de oluşan tabloları birleştir ve döndür.
    segment_statistics = pd.merge(segment_table, count_table, on="segment").sort_values("seg_count", ascending=False).round(2).reset_index(drop=True)
    return segment_statistics

In [ ]:
seg_stat = create_segment_statistic(rfm)
seg_stat

Segmentler incelendiğinde 2010-2011 yılları arasında en fazla sayıya sahip segment `hibernating`, en az sayıya sahip segment ise `new_customer` segmenti olduğu görülmektedir.

`champions` segmentinde **665** müşterimiz bulunmaktadır. Bu segmente ait olan bir müşteri; en son ortalama **9,3 gün önce** alışveriş yapmış, analiz gününe kadar ortalama **11,72 alışveriş yapmış** ve analiz gününe kadar yaptığı alışverişlerin toplamında bize ortalama **6288.26** sterlin para kazandırmış.

Diğer segmentler de bu şekilde yorumlanabilir.

In [ ]:
def abc_analysis_for_rfm(segment_statistics, abc_metrics="seg_count"):
    """
    RFM Analizi için ABC analizinin yapılmasını ve belirtilen metriğe göre segmentlerin A, B, C olarak gruplandırılmasını sağlayan fonksiyondur.
    
    Parameters
    -----------
    segment_statistics pd.DataFrame
        create_segment_statistic() fonksiyonundan oluşan segment istatistikleri tablosu.
    abc_metrics str, optional
        Varsayılan değeri "seg_count" olsa da ABC analizi metriği için "Recency", "Frequency" ve "Monetary" değişkenleri de belirlenebilir.
    
    Returns
    -----------
    segment_statistics pd.DataFrame
        segment_statistics tablosuna "ratio", "cum_ratio" ve "ABC Analysis" değişkenlerinin eklenmiş ve bütün ondalık sayıların 2 basamağa yuvarlandığı tablodur.
    
    Examples
    -----------
    >>> ...
    >>> analyse_date = dt.datetime(2011, 12, 11)
    >>> agg_dict = {"InvoiceDate": lambda date: (today_date - date.max()).days,
                    "Invoice": "nunique",
                    "TotalPrice": "sum"}
    >>> rfm = create_rfm_table(df, "Customer ID", agg_dict)
    >>> seg_stat = create_segment_statistic(rfm)
    >>> seg_stat
                       segment  Recency  Frequency  Monetary  seg_count
    0          hibernating   244.01       1.10    468.18       1069
    1      loyal_customers    42.17       6.82   3104.73        787
    2            champions     9.30      11.72   6288.26        665
    3              at_risk   171.67       2.83   1201.91        588
    4  potential_loyalists    20.34       2.03    846.57        478
    5       about_to_sleep    64.07       1.15    503.07        339
    6       need_attention    63.88       2.39   1087.42        196
    7            promising    28.71       1.00    327.78         96
    8           cant_loose   134.21       8.24   3551.16         68
    9        new_customers    11.15       1.00    342.55         52
    """
    col_list = list(segment_statistics.columns)
    col_list.remove(abc_metrics)
    col_list.append(abc_metrics)
    segment_statistics = segment_statistics[col_list].sort_values(abc_metrics, ascending=False)
    segment_statistics["ratio"] = segment_statistics[abc_metrics] / sum(segment_statistics[abc_metrics]) * 100
    segment_statistics["cum_ratio"] = np.cumsum(segment_statistics["ratio"])
    segment_statistics["ABC Analysis"] = segment_statistics["cum_ratio"].apply(lambda x: "A" if x < 81 else ("B" if x < 96 else "C"))
    return segment_statistics.round(2)

In [ ]:
abc_analysis_for_rfm(seg_stat, "Monetary")

Bu çalışmada "Hangi segmente nasıl odaklanmak gerekir?" sorusuna segment odaklı değil, müşterilerin toplam kazandırdıkları ücretlere göre ABC Analizi yapılarak cevap aranmaya çalışılmıştır. 

Bir müşterinin toplam ödediği ücrete bakılarak müşterilerimizin %80'ini `champions`, `cant_loose`, `loyal_customer`, `at_risk`; %15'ini `need_attention`, `potential_loyalists`, `about_to_sleep`; %5'ini ise `hibernating`, `new_customers` ve `promising` segmentlerinin oluşturulduğu görülmektedir. Bizim için önemli olan grup A grubu olduğundan A grubuna ait segmentler için aksiyon kararları verilebilir.



### Segmentlerin Görselleştirilmesi

In [ ]:
def create_segment_graph(rfm_table, abc_col, segment_labels="segment", graph_title="Online Retail II 2010-2011 RFM Segments"):
    """
    RFM analizi ve ABC analizinin bir arada bulunduğu Treemap grafiğini oluşturan fonksiyondur.
    Graifiği oluşturmak için plotly.express kütüphanesi kullanılmıştır.
    
    Parameters
    -----------
    rfm_table pd.DataFrame
        create_rfm_table() fonksiyonu kullanılarak oluşturulan RFM tablosu
    abc_col str, optional
        A, B, C gruplarını referans alacak değişkenin ismi
    segment_labels str, optional
        Segmentlerin bulunduğu değişkenin ismi
    graph_title str, optional
        Grafik ismi
    Returns
    -----------
    None
        Bu fonksiyon grafik gösterimi sağlar. Herhangi bir değer döndürmez.
    
    """
    rfm_table.sort_values(abc_col, ascending=False, inplace=True)
    rfm_table["ratio"] = rfm_table[abc_col] / sum(rfm_table[abc_col]) * 100
    rfm_table["cum_ratio"] = np.cumsum(rfm_table["ratio"])
    rfm_table["ABC Analysis"] = rfm_table["cum_ratio"].apply(lambda x: "A" if x < 81 else ("B" if x < 96 else "C"))
    rfm_table
    fig = px.treemap(rfm_table, path=["ABC Analysis", segment_labels], title=graph_title)
    fig.show()

In [ ]:
create_segment_graph(rfm, abc_col="Monetary")

Grafik incelendiğinde RFM tablosundaki `Monetary` değerleri ABC analizi için referans alınarak hazırlanan A, B, C sınıflarına göre segment bilgileri verilmiştir.

**Her bir müşterinin `Monetary` değerine göre:**

* B grubuna ait **1609** müşteri,
* C grubuna ait **1539** müşteri,
* A grubuna ait **1190** müşteri bulunmaktadır.

Her bir grubun alt gruplarında da segment sayıları görülmektedir. A grubuna odaklanıldığında beklenildiği gibi grubun neredeyse yarısını `champions` ve `loyal_customers` segmentleri oluşturmaktadır. Diğer gruplar ve segmentler bu şekilde yorumlanabilir.

## Kaynakça

* https://bootcamp.veribilimiokulu.com/egitim/veri-bilimci-yetistirme-programi/
* https://www.veribilimiokulu.com/rfm-analizi-ile-musteri-segmentasyonu/
* https://suleakcaycs.medium.com/rfm-anali%CC%87zi%CC%87-i%CC%87le-m%C3%BC%C5%9Fteri%CC%87-segmantasyonu-ve-veri%CC%87-seti%CC%87ni%CC%87-anlama-d809479caece
* https://barskavus.medium.com/rfm-ile-m%C3%BC%C5%9Fteri-segmentasyonu-2387669a8c86
* https://medium.com/analytics-vidhya/what-is-rfm-why-should-we-do-how-should-we-do-d0f09d7de5b5
* https://www.donusumdanismanlik.com/pareto-analizi-nedir/
* https://guillaume-martin.github.io/rfm-segmentation-with-python.html
* https://plotly.com/python/treemaps/